In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split

%matplotlib inline

1(+6). Проверить, сбалансирован ли датасет (может быть, наблюдений одного класса слишком много?). Какие результаты покажет dummy classifier, который будет всем новым наблюдениям присваивать класс ham? Насколько плохо такое решение для задачи определения спама?

Грубое решение - включить в training set только необходимое число наблюдений (примерно поровну spam и ham). 
Нормализовать тексты и обучить байесовскую модель (bag of words). Проверить, как влияют на результат:

1) разная токенизация: в одном случае знаки препинания удалять, в другом — считать их токенами;

2) лемматизация (отсутствие лемматизации, стемминг, лемматизация; инструменты можно использовать любые, например, nltk.stem);

3) удаление стоп-слов, а также пороги минимальной и максимальной document frequency;

4) векторизация документов (CountVectorizer vs. TfIdfVectorizer);

5) что-нибудь ещё?

При оценке классификатора обратите внимание на TP и FP.

Extra: ограничив количество наблюдений ham в обучающей выборке, мы игнорируем довольно много данных. 1) В цикле: случайно выбрать нужное число писем ham и сконструировать сбалансированную выборку, построить классификатор, оценить и записать результат; в итоге результаты усреднить. 2) поможет ли параметр class prior probability?

In [70]:
path = 'smsspamcollection/SMSSpamCollection'

messages = pd.read_csv(path, sep='\t',
                           names=["label", "message"])
messages.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Загрузили, посмотрели. Смотрим на классы

In [23]:
print(messages.groupby('label').describe())

                                                        message
label                                                          
ham   count                                                4825
      unique                                               4516
      top                                Sorry, I'll call later
      freq                                                   30
spam  count                                                 747
      unique                                                653
      top     Please call our customer service representativ...
      freq                                                    4


Выборка очевидно несбалансирована по классам (~6:1). Соответственно, dummy-classifier, всегда присваивающий ярлык "ham", будет иметь accuracy ~86%, а confusion matrix будет выглядеть вот так:

In [7]:
d = {'clf_ham': pd.Series([4825, 747], index=['ham', 'spam']),
    'clf_spam': pd.Series([0, 0], index=['ham', 'spam'])}
pd.DataFrame(d)

,clf_ham,clf_spam
ham,4825,0
spam,747,0


Для задачи определения спама такой классификатор абсолютно бесполезен просто по определению: это классификатор, который никогда не определяет спам

Соотношение 6:1 - вот и отлично, разделим ham на шесть непересекающихся выборок, обучим классификатор с каждым

In [43]:
# возьмём все сообщения ham (и все сообщения spam заодно)
ham = messages.loc[messages['label']=='ham']['message']
spam = messages.loc[messages['label']=='spam']['message']
ham.describe()

count                       4825
unique                      4516
top       Sorry, I'll call later
freq                          30
Name: message, dtype: object

In [82]:
# поделим на шесть равных выборок и запишем в датафрейм (игнорируем в итоге не больше пяти наблюдений)
ham_samples = pd.DataFrame(np.array([ham[i*6:i*6+6] for i in range(len(ham)//6)]))

ham_samples.describe()

,0,1,2,3,4,5
count,804,804,804,804,804,804
unique,793,783,782,798,790,785
top,"Sorry, I'll call later","Sorry, I'll call later",I cant pick the phone right now. Pls send a me...,I am in hospital da. . I will return home in e...,"Sorry, I'll call later","Sorry, I'll call later"
freq,5,7,5,2,4,9


Тут главное не пугаться одинаковых сообщений в разных выборках, это же топчик

In [89]:
# dummy tokenizer
def tokenize(text):
    text = text.lower()
    return word_tokenize(text)


ham_split = [ham_samples[i].map(tokenize) for i in range(6)]

ham_tok = pd.DataFrame(np.array(ham_split).T)
spam_tok = spam.map(tokenize)

print(spam_tok.head(1))
ham_tok.head(1)

2    [free, entry, in, 2, a, wkly, comp, to, win, f...
Name: message, dtype: object


,0,1,2,3,4,5
0,"[go, until, jurong, point, ,, crazy.., availab...","[ok, lar, ..., joking, wif, u, oni, ...]","[u, dun, say, so, early, hor, ..., u, c, alrea...","[nah, i, do, n't, think, he, goes, to, usf, ,,...","[even, my, brother, is, not, like, to, speak, ...","[as, per, your, request, 'melle, melle, (, oru..."


In [90]:
# теперь токенизация без знаков препинания
import string
def get_rid(arr):
    res = [x for x in arr if x not in string.punctuation]
    return 

SyntaxError: unexpected EOF while parsing (<ipython-input-90-3a143b86198b>, line 2)

In [ ]:
messages['length'] = messages['message'].map(lambda text: len(text))

In [ ]:
# print(messages.head())

# tokens = [word_tokenize(msg) for msg in messages]


def tokenize(text):
    text = text.lower()
    return word_tokenize(text)

# messages.message.head().apply(tokenize)
# messages.message = messages.message.apply(tokenize)

# print(messages.head())

In [ ]:
bow = CountVectorizer()
bow.fit_transform(messages['message'])
# print(bow.vocabulary_)

# m = messages['message'][3]
# print([m])
# bowed_m = bow.transform([m])
# print(bowed_m.shape)
# print(bow.get_feature_names()[1054])

bowed_messages = bow.transform(messages['message'])

In [ ]:
naive_model = MultinomialNB()
naive_model.fit(bowed_messages, messages['label'])

In [ ]:
# naive_model.predict()

# msg_train, msg_test, label_train, label_test = train_test_split(messages['message'], messages['label'], test_size=0.2)
# print(len(msg_train), len(msg_test))
cv_results = cross_val_score(naive_model, bowed_messages, messages['label'], cv=10, scoring='accuracy')
print(cv_results.mean(), cv_results.std())

In [ ]:
# pipeline = Pipeline([
#     ('bow', CountVectorizer(analyzer=tokenize)),
#     ('classifier', MultinomialNB()),
# ])
#
# cv_results = cross_val_score(pipeline,
#                              msg_train,
#                              label_train,
#                              cv=10,
#                              scoring='accuracy',
#                              )
# print(cv_results.mean(), cv_results.std())

2(+2). Сравнить результаты байесовского классификатора, решающего дерева и RandomForest. Помимо стандартных метрик оценки качества модели, необходимо построить learning curve, ROC-curve, classification report и интерпретировать эти результаты.

3(+2). А что, если в качестве предикторов брать не количество вхождений слов, а конструировать специальные признаки? Прежде всего, необходимо разделить таблицу на training set и test set в соотношении 80:20, test set не открывать до этапа оценки модели. С помощью pandas проверить, отличаются ли перечисленные ниже параметры (иможно придумать другие) для разных классов (spam/ham), и собрать матрицу признаков для обучения. Примеры признаков: длина сообщения, количество букв в ВЕРХНЕМ РЕГИСТРЕ, восклицательных знаков, цифр, запятых, каких-то конкретных слов (для этого можно построить частотный словарь по сообщениям каждого класса). Прокомментировать свой выбор. Векторизовать документы и построить классификатор. Оценить модель на проверочной выборке.

http://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/
http://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html